Microservices - Big Ball of Mud (Synchron)
-------------------------------

![](images/BigBallofMud.png)

- - -

Das Beispiel besteht aus Microservices: **Order**, **Customer**, **Catalog**, **Shipment**, **Invoicing** und **Sales**.

**Order** nutzt **Catalog** und **Customer** mittels deren REST-Schnittstelle. 
**Shipment**, **Invoicing** und **Sales** holen Daten von **Order**, **Catalog** und **Customer** mittels deren REST-Schnittstelle. 

Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Zusätzlich ist ein Service **Webshop** am laufen, der dem Benutzer mit einer Webseite einen einfachen Einstieg in das System ermöglicht.

- - -

Zuerst erstellen wir den Kubernetes Namespace ms-bbum und Labeln diesen, dass Kiali Informationen sammeln kann.

In [ ]:
! kubectl create namespace ms-bbum
! kubectl label  namespace ms-bbum istio-injection=enabled

Jetzt ist ein guter Zeitpunkt um das Kubernetes Dashboard zu starten und dort im Pulldownmenu den Namespace "ms-bbum" auszuwählen.

Wählt nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"

Der Webshop, aus den vorherigen Übungen, ist jetzt aufgeteilt in einzelne Microservices.

Diese starten wir deklarativ mittels `kubectl apply -f <Deklaration>`

In [ ]:
%%bash
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-deployment.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-deployment.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-service.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-service.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-service.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-service.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/shipment-deployment.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/invoicing-deployment.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/sales-deployment.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/shipment-service.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/invoicing-service.yaml
kubectl apply --namespace ms-bbum -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/sales-service.yaml
kubectl get pod,services --namespace ms-bbum

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-bbum webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

### Zipkin

Zipkin ist ein verteiltes tracing system. Es hilft beim Sammeln von Zeitdaten, die zur Behebung von Latenzproblemen in Service-Mesh erforderlich sind.  

In [ ]:
%%bash
kubectl get service -n istio-system -l name=zipkin -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
echo "Zipkin  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/zipkin -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

- - -
## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

In [ ]:
%%bash
URL="http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-bbum webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

for i in `seq 1 100`;
do
    hey -z 10s -c 10  ${URL}/order/order
    hey -z 10s -c 10  ${URL}/order/order
    hey -z 10s -c 10  ${URL}/shipment/shipment
    hey -z 10s -c 10  ${URL}/invoicing/invoicing
    hey -z 10s -c 10  ${URL}/sales/sales
    echo
done

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-bbum

- - -
### Quellen

* Sourcecode: https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v2.1.0?ref_type=heads
* Kubernetes Deklarationen: https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates
* Container Registry: https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/container_registry